## Pipes

Using LLM chain has been deprecated in new versions of Langchain. And Using pipes is recommended way to use chains.  
With Pipes, we can chain any number of components so that output of first component goes to input if next

### Setup LLM

In [1]:
from langchain_ollama.chat_models import ChatOllama

# Create chat ollama class to interact with ollama models
llm = ChatOllama(
    base_url = 'http://localhost:11434',
    model = 'qwen2.5:0.5b'
)

### Setup Prompt

In [2]:
from langchain.prompts import PromptTemplate

# This is simple Prompt template
prompt = PromptTemplate(template = """You are a super helpful health expert AI assistant. You can answer questions on health related topic.
                                    Answer the health related questions.
                                    If the question is not related to health, tell user that you can't answer this question.
                                    Keep the answer concise and maximum 2 sentences.
                                    Question: {question}
                                    Answer: """,
                       input_variables=['question'])

### Chain using Pipe

Now we can quickly create a chain using pipe method on Prompt and LLM

In [4]:
# Lets get StrOutput parser to extract string output from LLM response and chain it after llm
from langchain_core.output_parsers import StrOutputParser
chain = prompt.pipe(llm).pipe(StrOutputParser())

# Invoke Chain
question = input("Ask Question on Health: ").strip().strip('\n')
answer = chain.invoke({'question': question})
print(answer)

Ask Question on Health:  What is paracetamol?


Paracetamol is a pain reliever, fever reducer, and anti-inflammatory medication.


In [5]:
### Chains can also be piped using Langchain's Pipe operator
chain = prompt | llm | StrOutputParser()

# Invoke Chain
question = input("Ask Question on Health: ").strip().strip('\n')
answer = chain.invoke({'question': question})
print(answer)

Ask Question on Health:  What is diclofenac?


Diclofenac is a nonsteroidal anti-inflammatory drug (NSAID) used for treating arthritis pain and inflammation.


### Chaining Chains

Langchain also allow chaining the chains using pipe operator to make one Big chain

In [13]:
prompt1 = PromptTemplate(
    template="""Your are a medicine expert with knowledge about different type of medicines. 
                Users will provide some sysmptoms and your have to provide medicine recommendations for the symptoms. 
                Your answer should contain only name of medicine. 
                Here are Some Examples - 
                
                User: Body pain and fever
                AI: Paracetamol
                
                User: High acidity
                AI: Pantaprazole
                
                Now recommend medicine for following symptoms- 
                User: {symptoms}
                AI: """,
    input_variables = ['symptoms']
)

chain1 = prompt1 | llm | StrOutputParser()

symptoms = input("Enter your symptoms: ").strip().strip('\n')
recommendation = chain1.invoke({'symptoms': symptoms})
print(recommendation)

Enter your symptoms:  Pain in arms


Painkiller


In [17]:
prompt2 = PromptTemplate(
    template="""Your are a medicine expert with knowledge about different type of medicines and their side effects. 
                Users will provide medicine name and You need to provide side-effects of those medicines.
                Here are Some Examples - 
                
                User: Paracetamol
                AI: drowsiness, fatigue, rashes, and itching
                
                User: Pantaprazole
                AI: headaches and diarrhea
                
                Now provide the side effects of following medicines- 
                User: {medicine}
                AI: """,
    input_variables = ['medicine']
)

chain2 = prompt2 | llm | StrOutputParser()

medicine = input("Enter the mediocine name: ").strip().strip('\n')
side_effects = chain2.invoke({'medicine': medicine})
print(side_effects)

Enter the mediocine name:  diclofenac


I apologize for any confusion, but I can't determine the user's input with confidence. The provided information is incorrect. The most common side effect of diclofenac is gastrointestinal discomfort such as nausea and diarrhea.

However, if you provide me with the actual medicine name, I can assist in determining its known side effects. Please let me know which medicine you're referring to, and I'll be happy to provide you with accurate information about its side effects.


In [20]:
### Lets chain the 2 chains
chain3 = {'medicine': chain1} | chain2
symptoms = input("Enter your symptoms: ").strip().strip('\n')
side_effects = chain3.invoke({'symptoms': symptoms})
print(side_effects)

Enter your symptoms:  headache


Based on your examples, here are the side effects associated with ibuprofen:

1. Drowsiness - This is a common side effect of ibuprofen.
2. Fatigue - It can also cause tiredness and dizziness.
3. Rashes and itching - Some people may experience such symptoms after taking ibuprofen.
4. Headaches - While not typical, it's possible for some individuals to develop headaches when taking ibuprofen.
5. Diarrhea - This is another side effect that many people report after consuming ibuprofen.
6. Stomach upset - It can lead to stomach pain and discomfort.

These are just a few examples of the potential side effects of ibuprofen. Always consult with a healthcare professional before taking any new medication, especially if you're sensitive to it or have underlying health conditions.
